In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import math
def positionalEncodings(sequence):
    
    PE=[]
    d_model=64
    for pos in range(sequence):
        EncodingsOfEach=[]
        for i in range(d_model):
            if i%2==0:
                temp=pos/(math.pow(10000,(2*i)/d_model))
                EncodingsOfEach.append(math.sin(temp))
            else:
                temp=pos/(math.pow(10000,(2*(i-1))/d_model))
                EncodingsOfEach.append(math.cos(temp))
        PE.append(EncodingsOfEach)
    return PE

In [3]:
# def selfAttention(embeddings, W_q, W_k, W_v,mask):
#     batch_size = tf.shape(embeddings)[0]
#     seq_len = tf.shape(embeddings)[1]

#     batch_outputs = []

#     for b in range(batch_size):
#         x = embeddings[b]            # (seq_len, d_model)

#         Q, K, V = [], [], []
#         for i in range(seq_len):
#             x_i = tf.expand_dims(x[i], 0)   # (1, d_model)
#             Q.append(tf.matmul(x_i, W_q))
#             K.append(tf.matmul(x_i, W_k))
#             V.append(tf.matmul(x_i, W_v))

#         d_k = tf.math.sqrt(tf.cast(tf.shape(K[0])[1], tf.float32))

#         attention_outputs = []

#         for i in range(seq_len):
#             score_list = []
#             for j in range(seq_len):
#                 score = tf.matmul(Q[i], K[j], transpose_b=True)[0, 0] / d_k
#                 score=score+mask[i][j]
#                 score_list.append(score)

#             scores = tf.stack(score_list)
#             weights = tf.nn.softmax(scores)

#             output = tf.zeros_like(V[0])
#             for j in range(seq_len):
#                 output += tf.expand_dims(weights[j], axis=0) * V[j]

#             attention_outputs.append(output[0])

#         batch_outputs.append(tf.stack(attention_outputs))  # (seq_len, d_model)

#     return tf.stack(batch_outputs)  # (batch_size, seq_len, d_model)



In [4]:
def selfAttention(x, W_q, W_k, W_v):
    """
    x: (batch_size, seq_len, d_model)
    W_q, W_k, W_v: (d_model, d_model)
    """

    # Linear projections
    Q = tf.matmul(x, W_q)  # (batch, seq_len, d_model)
    K = tf.matmul(x, W_k)
    V = tf.matmul(x, W_v)

    d_k = tf.cast(tf.shape(K)[-1], tf.float32)

    # Scaled dot-product attention
    scores = tf.matmul(Q, K, transpose_b=True)  # (batch, seq_len, seq_len)
    scores = scores / tf.math.sqrt(d_k)

    weights = tf.nn.softmax(scores, axis=-1)    # (batch, seq_len, seq_len)

    output = tf.matmul(weights, V)               # (batch, seq_len, d_model)

    return output


In [5]:
layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
def Add_and_normalize(output_of_previous,input_of_previous):
    return layer_norm(output_of_previous + input_of_previous)

In [6]:
def FeedForwardNN(x, W1, b1, W2, b2):
    hidden = tf.nn.relu(tf.matmul(x, W1) + b1)
    out = tf.matmul(hidden, W2) + b2
    return out


In [7]:
def Encoder(positional_encodings_plus_embeddings,W_q,W_k,W_v,W1,b1,W2,b2):
    self_attn_output=selfAttention(positional_encodings_plus_embeddings,W_q,W_k,W_v)
    add_and_normalize_1_output=Add_and_normalize(self_attn_output,positional_encodings_plus_embeddings)
    ffn_output=FeedForwardNN(add_and_normalize_1_output,W1,b1,W2,b2)
    encoder_output=Add_and_normalize(ffn_output,add_and_normalize_1_output)
    return encoder_output

In [8]:
def encoderStack(x, encoder_params):
    for layer in encoder_params:
        x = Encoder(
            x,
            layer["W_q"],
            layer["W_k"],
            layer["W_v"],
            layer["W1"],
            layer["b1"],
            layer["W2"],
            layer["b2"]
        )
    return x


Decoder:


In [55]:
# def masked_Attention(embeddings, W_q, W_k, W_v,mask):
#     batch_size = tf.shape(embeddings)[0]
#     seq_len = tf.shape(embeddings)[1]

#     batch_outputs = []

#     for b in range(batch_size):
#         x = embeddings[b]            # (seq_len, d_model)

#         Q, K, V = [], [], []
#         for i in range(seq_len):
#             x_i = tf.expand_dims(x[i], 0)   # (1, d_model)
#             Q.append(tf.matmul(x_i, W_q))
#             K.append(tf.matmul(x_i, W_k))
#             V.append(tf.matmul(x_i, W_v))

#         d_k = tf.math.sqrt(tf.cast(tf.shape(K[0])[1], tf.float32))

#         attention_outputs = []

#         for i in range(seq_len):
#             score_list = []
#             for j in range(seq_len):
#                 score = tf.matmul(Q[i], K[j], transpose_b=True)[0, 0] / d_k
#                 score=score+mask[i][j]
#                 score_list.append(score)

#             scores = tf.stack(score_list)
#             weights = tf.nn.softmax(scores)

#             output = tf.zeros_like(V[0])
#             for j in range(seq_len):
#                 output += tf.expand_dims(weights[j], axis=0) * V[j]

#             attention_outputs.append(output[0])

#         batch_outputs.append(tf.stack(attention_outputs))  # (seq_len, d_model)

#     return tf.stack(batch_outputs)  # (batch_size, seq_len, d_model)

def masked_Attention(embeddings, W_q, W_k, W_v, mask):
    # embeddings: (B, T, D)
    # mask: (T, T)

    # 1. Linear projections
    Q = tf.matmul(embeddings, W_q)  # (B, T, D)
    K = tf.matmul(embeddings, W_k)  # (B, T, D)
    V = tf.matmul(embeddings, W_v)  # (B, T, D)

    # 2. Scaled dot-product attention scores
    d_k = tf.cast(tf.shape(K)[-1], tf.float32)
    scores = tf.matmul(Q, K, transpose_b=True)  # (B, T, T)
    scores = scores / tf.math.sqrt(d_k)
    mask = tf.cast(mask, tf.float32)
    scores += (1.0 - mask) * -1e9 

    # 3. Apply mask (broadcasted over batch)
     # (B, T, T)

    # 4. Softmax over last axis (keys)
    weights = tf.nn.softmax(scores, axis=-1)  # (B, T, T)

    # 5. Weighted sum of values
    output = tf.matmul(weights, V)  # (B, T, D)

    return output


In [10]:
# def Encoder_decoder_attention(encoder_output,prev_layer_output,W_q,W_k,W_v):
#     numberOfWords = tf.shape(encoder_output)[0]

#     Q, K, V = [], [], []
#     for i in range(numberOfWords):
#         q_i = tf.expand_dims(prev_layer_output[i], 0)
#         enc_i = tf.expand_dims(encoder_output[i], 0)
#         Q.append(tf.matmul(q_i, W_q))
#         K.append(tf.matmul(enc_i, W_k))
#         V.append(tf.matmul(enc_i, W_v))

#     d_k = tf.math.sqrt(tf.cast(tf.shape(K[0])[1], tf.float32))

#     attention_outputs = []

#     for i in range(numberOfWords):
#         score_list = []
#         for j in range(numberOfWords):
#             score = tf.matmul(Q[i], K[j], transpose_b=True)[0, 0] / d_k
#             score_list.append(score)

#         scores = tf.stack(score_list)
#         weights = tf.nn.softmax(scores)

#         output = tf.zeros_like(V[0])
#         for j in range(numberOfWords):
#             output += tf.expand_dims(weights[j], axis=0) * V[j]

#         attention_outputs.append(output[0])

#     return tf.stack(attention_outputs)

def Encoder_decoder_attention(
    encoder_output, prev_layer_output, W_q, W_k, W_v
):
    # encoder_output: (B, S, D)
    # prev_layer_output: (B, T, D)

    # 1. Projections
    Q = tf.matmul(prev_layer_output, W_q)  # (B, T, D)
    K = tf.matmul(encoder_output, W_k)     # (B, S, D)
    V = tf.matmul(encoder_output, W_v)     # (B, S, D)

    # 2. Scaled dot-product attention
    d_k = tf.cast(tf.shape(K)[-1], tf.float32)
    scores = tf.matmul(Q, K, transpose_b=True)  # (B, T, S)
    scores = scores / tf.math.sqrt(d_k)

    # 3. Softmax over encoder tokens
    weights = tf.nn.softmax(scores, axis=-1)  # (B, T, S)

    # 4. Weighted sum
    output = tf.matmul(weights, V)  # (B, T, D)

    return output


In [11]:
def decoder(positional_encodings_plus_embeddings,encoder_output,W_q_self,W_k_self,W_v_self,W_q_cross,W_k_cross,W_v_cross,mask,W1,b1,W2,b2):
    masked_self_attn_output=masked_Attention(positional_encodings_plus_embeddings,W_q_self,W_k_self,W_v_self,mask)
    add_and_normalize_1_output=Add_and_normalize(masked_self_attn_output,positional_encodings_plus_embeddings)
    encoder_decoder_attn_output=Encoder_decoder_attention(encoder_output,add_and_normalize_1_output,W_q_cross,W_k_cross,W_v_cross)
    add_and_normalize_2_output=Add_and_normalize(encoder_decoder_attn_output,add_and_normalize_1_output)
    ffn_output=FeedForwardNN(add_and_normalize_2_output,W1,b1,W2,b2)
    decoder_output=Add_and_normalize(ffn_output,add_and_normalize_2_output)
    return decoder_output

In [12]:
def causal_mask(d):
    mask = tf.linalg.band_part(tf.ones((d, d)), -1, 0)
    return (1.0 - mask) * -1e9


In [59]:
def decoderStack(x,decoder_params,encoder_output):
    seq_len = x.shape[1]       # current sequence length
    mask = causal_mask(seq_len)
    for layer in decoder_params:
        x=decoder(x,encoder_output,layer["W_q_self"],layer["W_k_self"],layer["W_v_self"],layer["W_q_cross"],layer["W_k_cross"],layer["W_v_cross"],mask,layer["W1"],layer["b1"],layer["W2"],layer["b2"])
    return x

Preparing the dataset:


In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [15]:
df=pd.read_csv('chatbot_dataset.csv')
df.head(1)

,User ID,User Utterance,Bot Response,Timestamp,Context/Session ID,Entities,User Feedback,Conversation Outcome,User Profile,Channel/Platform,Language,User Emotion/Sentiment,Location,User Segment
0,uKqYhMMQ5S,Charge bar between follow student.,Important law into large example range. Player...,2023-10-31 18:02:06,4dfe56dc-efe2-49c6-be9f-ce5b84ca4de4,event,negative,incomplete,Annette Henderson,social media,German,confused,Sydney,returning customers


In [16]:
x=df['User Utterance']
y=df['Bot Response']

In [17]:
x

0            Charge bar between follow student.
1               Bad every reflect huge contain.
2       Glass remember many dog director under.
3           Help charge record many talk tough.
4               Position not man much material.
                         ...                   
9995           How major might home gun reduce.
9996          Result executive some commercial.
9997                Require week affect factor.
9998                    Film I green like song.
9999                   Wonder majority million.
Name: User Utterance, Length: 10000, dtype: object

In [18]:
def clean_sentence(temp):
    temp=temp.lower()
    temp=temp.replace(".","")
    return temp

In [19]:
x=x.apply(clean_sentence)


In [20]:
x

0            charge bar between follow student
1               bad every reflect huge contain
2       glass remember many dog director under
3           help charge record many talk tough
4               position not man much material
                         ...                  
9995           how major might home gun reduce
9996          result executive some commercial
9997                require week affect factor
9998                    film i green like song
9999                   wonder majority million
Name: User Utterance, Length: 10000, dtype: object

In [21]:
y=y.apply(clean_sentence)

In [22]:
y_input="<sos> "+y
y_target=y+" <eos>"

In [23]:
from keras.preprocessing.text import Tokenizer

In [24]:
tokenizer=Tokenizer(filters='',oov_token="OOV")

In [25]:
tokenizer.fit_on_texts(list(x)+list(y_input)+list(y_target))


In [26]:
len(tokenizer.word_index)

1500

In [27]:
x=tokenizer.texts_to_sequences(x)


In [28]:
y_input=tokenizer.texts_to_sequences(y_input)
y_target=tokenizer.texts_to_sequences(y_target)

In [29]:
from tensorflow.keras.utils import pad_sequences
x=pad_sequences(x,padding='post')
y_input=pad_sequences(y_input,padding='post')
y_target=pad_sequences(y_target,padding='post')

In [30]:
numberOfWords=len(tokenizer.word_index)+1

In [31]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=numberOfWords,
    output_dim=64
)


In [32]:
x=embedding_layer(x)

In [33]:
y_input=embedding_layer(y_input)

In [34]:
PE=positionalEncodings(len(x[0]))

In [35]:
x=x+PE

In [36]:
PE1=positionalEncodings(len(y_input[0]))
y_input=y_input+PE1

In [37]:

d_model=x[0].shape[1]
encoder_params = []
for _ in range(6):
    params = {
        "W_q": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_k": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_v": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W1": tf.Variable(tf.random.normal(shape=(d_model,256))),
        "b1": tf.Variable(tf.random.normal(shape=(256,))),
        "W2": tf.Variable(tf.random.normal(shape=(256,d_model))),
        "b2": tf.Variable(tf.random.normal(shape=(d_model,)))
    }
    encoder_params.append(params)


In [38]:
encoder_output=(encoderStack(x,encoder_params))

In [39]:
d_model=y_input[0].shape[1]
numberOfWordsperSentence=y_input[0].shape[0]
mask=causal_mask(numberOfWordsperSentence)
decoder_params = []

for _ in range(6):
    params = {
        "W_q_self": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_k_self": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_v_self": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_q_cross": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_k_cross": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W_v_cross": tf.Variable(tf.random.normal(shape=(d_model,d_model))),
        "W1": tf.Variable(tf.random.normal(shape=(d_model,256))),
        "b1": tf.Variable(tf.random.normal(shape=(256,))),
        "W2": tf.Variable(tf.random.normal(shape=(256,d_model))),
        "b2": tf.Variable(tf.random.normal(shape=(d_model,)))
    }
    decoder_params.append(params)

In [58]:
print(decoderStack(y_input,decoder_params,encoder_output))

tf.Tensor(
[[[ 0.35350922  0.24897848  0.824047   ... -1.2552556   0.5592487
    1.2884918 ]
  [ 0.3534373   0.24865519  0.82367396 ... -1.2548338   0.5595525
    1.2884636 ]
  [ 0.35332078  0.24860422  0.8237599  ... -1.2544377   0.55886894
    1.2883502 ]
  ...
  [ 0.35370895  0.248875    0.8240993  ... -1.2548683   0.55925465
    1.2880447 ]
  [ 0.3536807   0.24893267  0.8241639  ... -1.2546885   0.5590889
    1.2881393 ]
  [ 0.35374263  0.24895956  0.8241448  ... -1.2546397   0.5590787
    1.2880994 ]]

 [[ 0.3685977   0.1748299   0.83255726 ... -1.3459191   0.5937771
    1.3169351 ]
  [ 0.36854485  0.17497113  0.8320781  ... -1.3460659   0.5939615
    1.3170239 ]
  [ 0.36858293  0.17512603  0.8322284  ... -1.3460875   0.59386307
    1.317037  ]
  ...
  [ 0.36861688  0.1748479   0.83251256 ... -1.3459136   0.59383476
    1.3167218 ]
  [ 0.3686169   0.17484792  0.83251256 ... -1.3459136   0.59383476
    1.3167219 ]
  [ 0.36861694  0.17484793  0.83251256 ... -1.3459135   0.59383476
 

In [41]:
def transformer_forward(x_input, y_input,
                        encoder_params, decoder_params,
                        W_out, b_out):

    encoder_output = encoderStack(x_input, encoder_params)
    decoder_output = decoderStack(y_input, decoder_params, encoder_output)
    logits = tf.einsum("btd,dk->btk", decoder_output, W_out) + b_out

    return logits  # (B, T, vocab_size)


In [42]:
vocab_size = numberOfWords # or whatever your tokenizer size is

W_out = tf.Variable(tf.random.normal((64, vocab_size)))
b_out = tf.Variable(tf.zeros((vocab_size,)))


In [43]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction="none"
)

def masked_loss(y_true, logits):
    # logits: (B, T, V)
    # y_true: (B, T)

    vocab_size = tf.shape(logits)[-1]

    # flatten
    logits = tf.reshape(logits, [-1, vocab_size])  # (B*T, V)
    y_true = tf.reshape(y_true, [-1])               # (B*T)

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y_true,
        logits=logits
    )

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask)



In [44]:
trainable_vars = []

for layer in encoder_params:
    trainable_vars.extend(layer.values())

for layer in decoder_params:
    trainable_vars.extend(layer.values())

trainable_vars.extend([W_out, b_out])


In [45]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-4,
    beta_1=0.9,
    beta_2=0.98,
    epsilon=1e-9
)


In [46]:
@tf.function
def train_step(x_input, y_input, y_target):
    with tf.GradientTape() as tape:
        logits = transformer_forward(
            x_input, y_input,
            encoder_params, decoder_params,
            W_out, b_out
        )
        loss = masked_loss(y_target, logits)

    grads = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(grads, trainable_vars))
    return loss


In [47]:
BATCH_SIZE = 32
EPOCHS = 10

num_samples = x.shape[0]

for epoch in range(EPOCHS):
    total_loss = 0.0
    steps = 0

    for i in range(0, num_samples, BATCH_SIZE):
        x_batch = x[i:i+BATCH_SIZE]
        y_in_batch = y_input[i:i+BATCH_SIZE]
        y_tgt_batch = y_target[i:i+BATCH_SIZE]

        # skip last incomplete batch (optional)
        if x_batch.shape[0] < BATCH_SIZE:
            continue

        loss = train_step(x_batch, y_in_batch, y_tgt_batch)
        total_loss += loss
        steps += 1

    print(f"Epoch {epoch+1}, Loss: {total_loss / steps:.4f}")


Epoch 1, Loss: 17.5515
Epoch 2, Loss: 14.0488
Epoch 3, Loss: 12.2820
Epoch 4, Loss: 10.8817
Epoch 5, Loss: 9.7726
Epoch 6, Loss: 8.9350
Epoch 7, Loss: 8.3348
Epoch 8, Loss: 7.9113
Epoch 9, Loss: 7.6021
Epoch 10, Loss: 7.3685


In [49]:
logits = transformer_forward(
    x[:32],
    y_input[:32],
    encoder_params,
    decoder_params,
    W_out,
    b_out
)

print(logits.shape)


(32, 19, 1501)


In [50]:
print(y_input.shape)   # (B, T)
print(y_target.shape)  # (B, T)


(10000, 19, 64)
(10000, 19)


In [51]:
print(x.shape)
print(y_input.shape)   
print(y_target.shape) 

(10000, 10, 64)
(10000, 19, 64)
(10000, 19)


In [52]:
encoder_params[0]["W_q"].numpy()


array([[-0.05736803,  0.5728662 , -0.4638267 , ...,  0.38858372,
         0.05703561,  1.0867084 ],
       [-0.7731334 , -1.1273875 ,  0.87432116, ...,  1.1872966 ,
        -0.16804042,  2.6246243 ],
       [ 1.0465385 ,  0.37476498,  0.11576691, ...,  1.3860431 ,
        -1.3612062 , -0.942404  ],
       ...,
       [ 0.2539682 , -1.7078447 ,  0.7882006 , ...,  1.7670351 ,
         1.3164395 , -0.97455955],
       [-0.6544962 , -0.2958214 , -0.14303397, ..., -1.6947759 ,
         0.81154436, -1.305567  ],
       [-1.165641  , -0.5689572 , -0.50036865, ..., -0.307711  ,
         0.4265408 , -0.6049645 ]], dtype=float32)

In [60]:
def greedy_decode(
    input_sentence,
    max_len=30
):
    
    x_ids = tokenizer.texts_to_sequences([input_sentence])
    x_ids=pad_sequences(x_ids,padding='post',maxlen=10)
    x_ids = tf.constant(x_ids)

    x_embed = embedding_layer(x_ids)
    x_embed += positionalEncodings(x_embed.shape[0])

    encoder_out = encoderStack(x_embed, encoder_params)

    # 2. Start decoder with <sos>
    sos_id = tokenizer.word_index["<sos>"]
    eos_id = tokenizer.word_index["<eos>"]

    y_ids = tf.constant([[sos_id]])

    for _ in range(max_len):
        y_embed = embedding_layer(y_ids)
        y_embed += positionalEncodings(y_embed.shape[0])

        decoder_out = decoderStack(y_embed, decoder_params, encoder_out)

        logits = tf.einsum("btd,dk->btk", decoder_out, W_out) + b_out

        next_token = tf.argmax(logits[:, -1, :], axis=-1).numpy()[0]

        if next_token == eos_id:
            break

        y_ids = tf.concat(
            [y_ids, [[next_token]]],
            axis=1
        )

    return tokenizer.sequences_to_texts(y_ids.numpy())[0]


In [66]:
print(greedy_decode("Hello what are you doing and how is life and the moon is the only light"))

<sos>


In [67]:
x_ids = tokenizer.texts_to_sequences(["Hello what are you doing now and how is life"])
x_ids=pad_sequences(x_ids,padding='post',maxlen=10)
x_ids = tf.constant(x_ids)

In [187]:
x_ids

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[  1, 115,   1,   4,   1,   0,   0,   0,   0,   0]])>

In [188]:
x_embed = embedding_layer(x_ids)

In [199]:
iric=positionalEncodings(x_embed.shape[0])

In [200]:
x_embed=x_embed+iric

In [201]:
x_embed

<tf.Tensor: shape=(1, 10, 64), dtype=float32, numpy=
array([[[ 2.75616758e-02,  9.66541171e-01,  4.55960520e-02,
          9.66299355e-01, -2.42324919e-03,  1.03603590e+00,
          1.65901519e-02,  9.63546216e-01, -3.89556773e-02,
          1.03848183e+00,  1.14405751e-02,  9.80966210e-01,
          4.35198434e-02,  9.51243818e-01,  9.47365910e-03,
          9.83551204e-01, -4.39269543e-02,  1.04676902e+00,
          3.15343849e-02,  9.55221474e-01, -4.52478416e-02,
          1.04918134e+00, -4.25847657e-02,  9.51684892e-01,
          2.84588002e-02,  1.02075958e+00, -2.52927430e-02,
          1.03773868e+00, -3.01157963e-02,  9.86772180e-01,
         -4.07280326e-02,  1.03867781e+00,  4.87259068e-02,
          9.55558717e-01, -1.57153383e-02,  1.00135493e+00,
          3.52635719e-02,  1.03862929e+00,  3.62424962e-02,
          9.68588650e-01, -6.80303574e-03,  9.76093829e-01,
         -4.53583859e-02,  9.70077932e-01, -2.27234364e-02,
          9.91288662e-01, -4.89297882e-02,  1.0

In [202]:
encoderStack(x_embed,encoder_params)

<tf.Tensor: shape=(1, 10, 64), dtype=float32, numpy=
array([[[-3.8402182e-01,  3.8071486e-01, -1.4521152e+00, -8.0156282e-02,
         -2.5665715e-01,  8.9871258e-01,  1.4151897e+00, -2.9854476e-03,
          1.7579795e+00, -9.0312648e-01,  4.4051975e-01, -1.3489048e+00,
          1.1900598e+00,  1.4108388e-01,  4.6423602e-01, -1.1340650e+00,
         -8.4874266e-01, -1.9558892e+00, -1.2075388e+00, -7.4558467e-01,
          3.2968691e-01,  1.3915409e+00,  2.5755930e-01,  9.5502484e-01,
          7.6846975e-01, -1.1092676e-01,  1.4209023e-01, -3.3388066e-01,
         -3.3443296e-01, -5.5233955e-01,  7.5723544e-02,  1.6536520e+00,
         -4.3941504e-01,  5.7350218e-01, -1.2600713e+00, -3.7725902e-01,
          3.0341351e-01, -1.4200146e+00,  5.6888777e-01,  2.2883797e+00,
          1.4143739e+00, -6.8856668e-01,  7.3873222e-01, -5.3413892e-01,
          1.8211945e+00, -2.2991548e+00, -1.0314559e+00, -1.9169524e-03,
         -2.9897302e-01,  1.8312907e-01, -1.7014435e-01, -8.4164613e-01

In [203]:
sos_id = tokenizer.word_index["<sos>"]

In [204]:
sos_id

2